In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [15]:
restaurants_df = pd.read_csv('restaurants.csv')

In [16]:
#drop Unnamed:0 column
restaurants_df_drop_col = restaurants_df.drop(['Unnamed: 0'], axis=1)

In [17]:
#remove duplicate rows :
restaurants_df_drop_col_remove_dups = restaurants_df_drop_col.drop_duplicates(keep = False)

In [19]:
#Top 10 indian restaurants in Toronto, number of reviews, then number of stars
top_ten_rests = restaurants_df_drop_col_remove_dups.sort_values(['restaurant_reviews_count','star_rating'], ascending = (False,False)).head(10)
top_ten_rests



,rest_name,star_rating,restaurant_reviews_count,restaurant_price,restaurant_cuisines,restaurant_area,restaurant_delivery_options
72,Byblos,4.5,814,$$$,"['Mediterranean', 'Middle Eastern']",Entertainment District,"['Delivery', 'Takeout']"
185,Maha’s,4.0,522,$$,"['Egyptian', 'Vegan', 'Sandwiches']",Leslieville,"['Outdoor dining', 'Takeout']"
28,Lahore Tikka House,3.5,508,$$,"['Halal', 'Pakistani']",Leslieville,"['Outdoor dining', 'Delivery', 'Takeout']"
17,Banjara Indian Cuisine,3.5,417,$$,['Indian'],Christie Pits,[]
255,Fresh on Spadina,4.0,406,$$,"['Vegan', 'Juice Bars & Smoothies', 'Burgers']",Entertainment District,"['Proof of vaccination required', 'Takeout']"
21,Little India Restaurant,3.5,390,$$,"['Indian', 'Buffets']",Queen Street West,"['Delivery', 'Takeout']"
292,The Halal Guys,3.0,257,$$,"['Middle Eastern', 'Halal']",Church-Wellesley Village,"['Delivery', 'Takeout']"
57,Tibet Kitchen,4.5,245,$$,['Himalayan/Nepalese'],Parkdale,['Delivery']
36,Aroma Fine Indian Cuisine,3.5,210,$$,['Indian'],Entertainment District,['Delivery']
15,Indian Roti House,4.0,208,$$,['Indian'],Harbourfront,"['Delivery', 'Takeout']"


In [20]:
most_popular_rest = top_ten_rests.iloc[3,:]

In [7]:
most_popular_rest

rest_name                      Banjara Indian Cuisine
star_rating                                       3.5
restaurant_reviews_count                          417
restaurant_price                                   $$
restaurant_cuisines                        ['Indian']
restaurant_area                         Christie Pits
restaurant_delivery_options                        []
Name: 17, dtype: object

In [119]:
#rest_name_area_lower = most_popular_rest['rest_name'].lower() + ' ' + most_popular_rest['restaurant_area'].lower()
#rest_name_area_lower

In [120]:
#rest_name_area_lower_list = rest_name_area_lower.split()
#rest_name_area_lower_list

In [121]:
#rest_name_lower = most_popular_rest['rest_name'].lower()
#rest_name_lower

In [122]:
#rest_name_list = rest_name_lower.split()
#rest_name_list

In [118]:
#x = '-'.join(rest_name_list)
#x

In [117]:
#y = '+'.join(rest_name_area_lower_list)
#y

In [116]:
#m = 'https://www.yelp.ca/biz/' + x + 'toronto?osq=' + y
#m

In [115]:
#rest_name_lower = most_popular_rest['rest_name'].lower()
#print(rest_name_lower)

#rest_name_area_lower = most_popular_rest['rest_name'].lower() + ' ' + most_popular_rest['restaurant_area'].lower()
#print(rest_name_area_lower)


In [21]:
#returns url of most popular restauramt

def return_url(rest_name_lower, rest_name_area_lower):
    
    rest_name_area_lower_list = rest_name_area_lower.split()
    rest_name_list = rest_name_lower.split()
    
    x = '-'.join(rest_name_list)
    y = '+'.join(rest_name_area_lower_list) 
   
    main_url_str = 'https://www.yelp.ca/biz/' + x + '-toronto?osq=' + y
    
    return f'{main_url_str}'
    

In [114]:
#return_url('banjara indian cuisine','banjara indian cuisine christie pits')

In [ ]:
#https://www.yelp.ca/biz/banjara-indian-cuisine-christie-pitstoronto?osq=banjara+indian+cuisine

In [ ]:
#start technical analysis of most popular indian restaurant in Toronto

In [24]:
import requests 
driver = webdriver.Chrome()

#get url of restaurant page :
rest_name_lower = most_popular_rest['rest_name'].lower()
rest_name_area_lower = most_popular_rest['rest_name'].lower() + ' ' + most_popular_rest['restaurant_area'].lower()

page_url = return_url(rest_name_lower,rest_name_area_lower)
try :
    response = requests.get(page_url)
    code = response.status_code 
    if code == 200 :  
        driver.get(page_url)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)
except :
    print('Bad request')
    
    
    
    

In [51]:
#most popular dishes 
def get_famous_dishes_info():
    
    dict_dishes = {
        'dish_name': [],
        'number_photos': [],
        'number_reviews' : []
    }
    
    #refers to all dishes on
    containers = soup.find_all('div', class_ = 'dishPassport__09f24__dapXe border-color--default__09f24__NPAKY background-color--white__09f24__ulvSM')
    for container in containers :
        
        #get name of dishes :
        try :
            dish_name = container.find('p', class_ = 'css-1peqmen').text
        except :
            dish_name = ''
            
        dict_dishes['dish_name'].append(dish_name)
        
        #get number of photos :
        try :
            num_photos_str = container.find('span', class_ = 'css-1e4fdj9').text #str
            clean_num_photos_str = unicodedata.normalize("NFKD",num_photos_str) #remove xao from string
            num_photos_int = int(clean_num_photos_str.split()[0])
        except :
            num_photos_int = 0
        
        dict_dishes['number_photos'].append(num_photos_int)
        
        #get number of reviews
        try :
            num_reviews_str = container.find('span', class_ = 'css-hvzf3i').text
            num_reviews_int = int(num_reviews_str.split()[0]) 
        except :
            num_reviews_int = 0
        
        dict_dishes['number_reviews'].append(num_reviews_int)
        df = pd.DataFrame(dict_dishes)
            
    return df

In [76]:
famous_dishes_df = get_famous_dishes_info()


In [77]:
#move data from df to csv
famous_dishes_df.to_csv('famous_dighes_popular_rest.csv')

In [79]:
#move data from csv back to a df
dish_df = pd.read_csv('famous_dighes_popular_rest.csv')
dish_df

,Unnamed: 0,dish_name,number_photos,number_reviews
0,0,Butter Chicken,22,119
1,1,Palak Paneer,2,24
2,2,Aloo Gobi,1,22
3,3,Tandoori Chicken,4,13
4,4,Chicken Korma,1,14
5,5,Malai Kofta,1,15
6,6,Chicken Tikka Masala,1,12
7,7,Paneer Makhani,3,10
8,8,Basmati Rice,1,5
9,9,Lamb Rogan Josh,1,6


In [81]:
#drop Unnamed:0 column
dish_df_drop_0col = dish_df.drop(['Unnamed: 0'], axis=1)
dish_df_drop_0col

,dish_name,number_photos,number_reviews
0,Butter Chicken,22,119
1,Palak Paneer,2,24
2,Aloo Gobi,1,22
3,Tandoori Chicken,4,13
4,Chicken Korma,1,14
5,Malai Kofta,1,15
6,Chicken Tikka Masala,1,12
7,Paneer Makhani,3,10
8,Basmati Rice,1,5
9,Lamb Rogan Josh,1,6


In [82]:
#top 5 dishes
dish_df_drop_0col.head()

,dish_name,number_photos,number_reviews
0,Butter Chicken,22,119
1,Palak Paneer,2,24
2,Aloo Gobi,1,22
3,Tandoori Chicken,4,13
4,Chicken Korma,1,14


In [50]:
#containers = soup.find_all('div', class_ = 'dishPassport__09f24__dapXe border-color--default__09f24__NPAKY background-color--white__09f24__ulvSM')
#container0 = containers[0]
#print(container0)

In [53]:
#dish_name = container0.find('p', class_ = 'css-1peqmen').text
#dish_name


'Butter Chicken'

In [49]:
import unicodedata

#num_photos_str = container0.find('span', class_ = 'css-1e4fdj9').text
#clean_num_photos_str = unicodedata.normalize("NFKD",num_photos_str)
#int(clean_num_photos_str.split()[0])

In [47]:
#num_reviews = container0.find('span', class_ = 'css-hvzf3i').text
#int(num_reviews.split()[0])

In [37]:
'''
containers = soup.find_all('div', class_ = 'review__09f24__oHr9V border-color--default__09f24__NPAKY')
container0 = containers[0]
#print(container0)

'''




"\ncontainers = soup.find_all('div', class_ = 'review__09f24__oHr9V border-color--default__09f24__NPAKY')\ncontainer0 = containers[0]\n#print(container0)\n\n"

In [2]:
'''
reviewer_name = container0.find('a', class_ = 'css-1422juy').text
reviewer_name
'''


"\nreviewer_name = container0.find('a', class_ = 'css-1422juy').text\nreviewer_name\n"

In [41]:

#location of reviewer
#reviewer_location = container0.find('span', class_ = 'css-1sufhje').text
#reveiwer_city = reviewer_location.split(',')[0]
#reveiwer_city


In [4]:
'''
#star rating

review_star_rating_str = container0.find('div', class_='i-stars__09f24__foihJ i-stars--regular-5__09f24__fJwXW border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG').get('aria-label')
review_star_rating_flt = float(review_star_rating_str.split()[0])
review_star_rating_flt
'''


"\n#star rating\n\nreview_star_rating_str = container0.find('div', class_='i-stars__09f24__foihJ i-stars--regular-5__09f24__fJwXW border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG').get('aria-label')\nreview_star_rating_flt = float(review_star_rating_str.split()[0])\nreview_star_rating_flt\n"

In [84]:
'''
#date of review
review_date_str = container0.find('span', class_ = 'css-1e4fdj9').text
review_date_str
'''

'11/28/2021'

In [5]:
'''
review_date_formatted = pd.to_datetime(review_date_str,format = '%m/%d/%Y')
review_date_formatted
'''

"\nreview_date_formatted = pd.to_datetime(review_date_str,format = '%m/%d/%Y')\nreview_date_formatted\n"

In [113]:
#type(review_date_formatted)

pandas._libs.tslibs.timestamps.Timestamp

In [6]:
'''
#review text :
review_text = container0.find('span', class_ = 'raw__09f24__T4Ezm').text
review_text
'''


"\n#review text :\nreview_text = container0.find('span', class_ = 'raw__09f24__T4Ezm').text\nreview_text\n"

In [7]:
'''
#useful, funny, cool
container4 = containers[4]
print(container4)
'''

'\n#useful, funny, cool\ncontainer4 = containers[4]\nprint(container4)\n'

In [8]:
'''
notes = container4.find_all('span', class_ = 'css-1rw3tz3')
print(notes)
'''


"\nnotes = container4.find_all('span', class_ = 'css-1rw3tz3')\nprint(notes)\n"

In [9]:
#len(notes)

In [10]:

'''
note = notes[0]
note.text
'''


'\nnote = notes[0]\nnote.text\n'

In [11]:
'''
note1 = notes[1]
note1.text
'''


'\nnote1 = notes[1]\nnote1.text\n'

In [12]:
'''
note2 = notes[2]
note2.text
'''

'\nnote2 = notes[2]\nnote2.text\n'

In [13]:
'''
for note in notes:
    print(note.text)
    print(int(note.text.split()[1]))
'''

'\nfor note in notes:\n    print(note.text)\n    print(int(note.text.split()[1]))\n'

In [34]:
#returns all information about the reviews on 1st page, in a dict format
def get_review_info():
    
    review_dict = {
        'review_text':[],
        'review_date':[],
        'review_star_rating': [],
        'reviewer_city': [],
        'review_useful_count':[],
        'review_funny_count':[],
        'review_cool_count':[]
    }
    
    #containers contain all reviews on page 1, one container for each review
    containers = soup.find_all('div', class_ = 'review__09f24__oHr9V border-color--default__09f24__NPAKY')

    for container in containers :
        #get review text :
        try :
            comment = container.find('span', class_ = 'raw__09f24__T4Ezm').text 
        except :
            comment = '' 
        review_dict['review_text'].append(comment)
        
        #get review date :
        try:
            review_date_str = container.find('span', class_ = 'css-1e4fdj9').text
            review_date_formatted = pd.to_datetime(review_date_str,format = '%m/%d/%Y')   
        except :
            review_date_formatted = ''
        review_dict['review_date'].append(review_date_formatted)
        
        #get star rating
        try:
            review_star_rating_str = container.find('span',class_='display--inline__09f24__c6N_k border-color--default__09f24__NPAKY').div['aria-label']
            review_star_rating_flt = float(review_star_rating_str.split()[0]) 
        except :
            review_star_rating_flt = 0.0 
        review_dict['review_star_rating'].append(review_star_rating_flt)
        
        #get reviewer location 
        try:
            reviewer_location = container.find('span', class_ = 'css-1sufhje').text 
            reviewer_city = reviewer_location.split(',')[0]
        except:
            reviewer_city = ''
        review_dict['reviewer_city'].append(reviewer_city)
        
        #get emotions number
        notes = container.find_all('span', class_ = 'css-1rw3tz3')
        try:
            useful_count =  int(notes[0].text.split()[1])
        except :
             useful_count = 0
        review_dict['review_useful_count'].append(useful_count)
        
        try:
            funny_count =  int(notes[1].text.split()[1]) 
        except :
            funny_count = 0 
        review_dict['review_funny_count'].append(funny_count)
        
        try:
            cool_count =  int(notes[2].text.split()[1]) 
        except :
            cool_count = 0 
        review_dict['review_cool_count'].append(cool_count)
        
        
    return review_dict

In [35]:
get_review_info()

{'review_text': ['Went here for my first meal in Toronto and it did not disappoint. I got a veggie combo- it came with palak paneer, aloo gobi, plain naan, rice, onion pakora, daal, and rice pudding. Such a great deal for a low cost, and it easily was enough food to split into 2-3 meals.',
  "For the life of me I really don't understand why Banjara is as highly rated as it is.The food is BLAND. even medium spice level is more mild than any other Indian food restaurant I've ever been to. The entrees have almost no sauce, it's just meat and/or veg and a tiny amount of sauce. Want a potato curry? Ok here is a bunch of large hard pieces of potato in a tiny amount of sauce. I hadn't had Banjara in a few years and ordered some last night and it's basically the most disappointing Indian food I've had in years. So bland, so boring, not savoury, not rich in flavour, and expensive also! Never again. Only giving 2 stars because at leas the veg pakoras were good.",
  "I love Banjara! It's impressi

In [25]:
'''
containers = soup.find_all('div', class_ = 'review__09f24__oHr9V border-color--default__09f24__NPAKY')
container0 = containers[0]
container0.find('div', class_='i-stars__09f24__foihJ i-stars--regular-5__09f24__fJwXW border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG').get('aria-label')

'''


"\ncontainers = soup.find_all('div', class_ = 'review__09f24__oHr9V border-color--default__09f24__NPAKY')\ncontainer0 = containers[0]\ncontainer0.find('div', class_='i-stars__09f24__foihJ i-stars--regular-5__09f24__fJwXW border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG').get('aria-label')\n\n"

In [26]:
# star_rating = container.find('span',class_ = 'display--inline__09f24__c6N_k border-color--default__09f24__NPAKY').div.get('aria-label')
#containers = soup.find_all('div', class_ = 'review__09f24__oHr9V border-color--default__09f24__NPAKY')
#container1 = containers[1]
#print(container1)
#container1.find('span', class_='display--inline__09f24__c6N_k border-color--default__09f24__NPAKY').div.get('aria-label')


In [27]:
#container2 = containers[2]
#container2.find('span', class_='display--inline__09f24__c6N_k border-color--default__09f24__NPAKY').div.get('aria-label')

In [28]:
#container2 = containers[2]
#container2.find('span', class_='display--inline__09f24__c6N_k border-color--default__09f24__NPAKY').div['aria-label']
    
    